# Gene analysis utility functions

This relies on the `Header` notebook already having been run. The `gene` objects here are named tuples from `parse_sco().itertuples()`.

... TODO, this should really just be a python module, sheesh

In [2]:
def get_pos_interval_from_gene(gene):
    try:
        start = int(gene.LeftEnd)
    except ValueError:
        # Account for the "<2" left end coordinate in one of the genes predicted
        start = int(gene.LeftEnd[1:])
        
    try:
        end = int(gene.RightEnd)
    except ValueError:
        # Account for the ">N" right end coordinate in one of the genes predicted
        end = int(gene.RightEnd[1:])

    if gene.Strand == "-":
        return range(end, start - 1, -1)
    elif gene.Strand == "+":
        return range(start, end + 1)
    else:
        raise ValueError("Unknown strand: {}".format(gene.Strand))

In [1]:
def validate_gene_coords(gene):
    if gene.LeftEnd >= gene.RightEnd:
        raise ValueError(
            f"Gene {gene.Index}'s coordinates seem messed up: "
            f"left = {gene.LeftEnd:,}, right = {gene.RightEnd:,}"
        )

In [6]:
def get_gene_left_codon_positions(gene):
    return [
        i for i in range(gene.LeftEnd, gene.RightEnd + 1, 3)
    ]

In [12]:
# TODO: Should really move this to another file, automate this process with Travis / GH Actions, etc.
def test():
    import parse_sco
    camp_genes = parse_sco.parse_sco("../seqs/genes/edge_1671.sco")
    for gene in camp_genes.itertuples():
        validate_gene_coords(gene)
        left_cps = get_gene_left_codon_positions(gene)
        assert len(left_cps) == (gene.RightEnd - gene.LeftEnd + 1) / 3
# test()